# Example `KBackground.Estimator` workflow with Kepler TPFs

In [1]:
import lightkurve as lk
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
from kbackground import Estimator
import astropy.units as u

In [2]:
# This let's us have debugging messages!
import kbackground
kbackground.log.setLevel("DEBUG")

First we are going to get a number of the TPFs in a large superstamp in campaign 8. This will let us see a patch of data with a rolling band.

In [3]:
tpfs = lk.search_targetpixelfile('KIC 8557374', quarter=5, cadence='long', radius=10 * u.arcmin).download_all()

Next we will extract the row, column and flux values from all the TPFs.

In [4]:
rows, columns, fluxs, mission_bkgs, cadencenos = [], [], [], [], []
for tpf in tpfs:
    column, row = np.meshgrid(np.arange(tpf.column, tpf.shape[2] + tpf.column), np.arange(tpf.row, tpf.shape[1] + tpf.row))
    aper = np.nansum(tpf.flux.value, axis=0) != 0
    rows.append(row[aper])
    columns.append(column[aper])
    fluxs.append(tpf.flux.value[:, aper] + np.nan_to_num(tpf.flux_bkg.value[:, aper]))
    cadencenos.append(tpf.cadenceno)

Next we can build the `Estimator` object, passing in the row, column, and flux values from all the TPFs.

In [5]:
r, c, f = np.hstack(rows), np.hstack(columns), np.hstack(fluxs)
k = np.isfinite(f).all(axis=0)

In [6]:
e = Estimator(cadencenos[0], np.hstack([r[k] - 170, r[k], r[k] + 170]), np.hstack([c[k]] * 3), np.hstack([f[:, k], f[:, k], f[:, k]]))

2022-01-20 18:22:27,652 - kbackground.kbackground - DEBUG - new `kbackground` Object. tknotspacing:10, xknotspacing:10
2022-01-20 18:22:27,653 - kbackground.kbackground - DEBUG - ntimes x npixels : (4492, 6582)
2022-01-20 18:22:29,276 - kbackground.kbackground - DEBUG - unq_row : 705 ... 1212 (444 unique rows)
2022-01-20 18:22:29,277 - kbackground.kbackground - DEBUG - Binning flux data
2022-01-20 18:22:29,352 - kbackground.kbackground - DEBUG - Binned flux data
2022-01-20 18:22:29,352 - kbackground.kbackground - DEBUG - `bf` (444, 4492)
2022-01-20 18:22:29,352 - kbackground.kbackground - DEBUG - Making A
2022-01-20 18:22:29,360 - kbackground.kbackground - DEBUG - Made `x_spline` (444, 25)
2022-01-20 18:22:29,558 - kbackground.kbackground - DEBUG - Made `t_spline` (4492, 466)
2022-01-20 18:22:30,156 - kbackground.kbackground - DEBUG - Made `X` (1994448, 25)
2022-01-20 18:22:30,169 - kbackground.kbackground - DEBUG - Made `T` (1994448, 466)
2022-01-20 18:22:33,564 - kbackground.kbackgro

In [ ]:
e.plot();

### Correction for faint pixels

Below we show the correction for a subset of the faint pixels in this dataset. The correction largely removes the long term trends from the dataset.

In [ ]:
dim_pix = np.where(np.diff(np.nanpercentile(e.flux, (5, 95), axis=0), axis=0)[0] < 80)[0]
fig, ax = plt.subplots(4, 4, figsize=(8, 8), sharex=True, sharey=True)
kdx = -1
for idx, jdx in enumerate(np.random.choice(dim_pix, 16)):
    if idx % 4 == 0:
        kdx += 1
    ax[kdx, idx % 4].plot(tpf.time.value, e.flux[:, jdx], label='Data', c='grey')
    ax[kdx, idx % 4].plot(tpf.time.value, e.flux[:, jdx] - e.model[:, jdx], label='Data', c='k')
    ax[kdx, idx % 4].plot(tpf.time.value, e.model[:, jdx], label='Model', c='r')
    ax[kdx, idx % 4].set(ylim=(-100, ax[kdx, idx % 4].get_ylim()[1]))